# CartPole

In [7]:
import gym
import random
import numpy as np
import pandas as pd
from pylab import plt
from IPython import display
plt.style.use('seaborn-v0_8')
np.random.seed(100)
import warnings; warnings.simplefilter('ignore')

import tensorflow.compat.v1 as tf
from keras. layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
tf.random.set_random_seed(100)
tf.logging.set_verbosity(tf.logging.ERROR)

env = gym.make ( 'CartPole-v0')

# Setting Seeds

In [8]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    env.seed(seed)

# Q Learning

In [9]:
from collections import deque

class DQNAgent:
  def __init__(self, finish=False):
    self.finish = finish
    self.epsilon = 1.0
    self.epsilon_min = 0.01
    self.epsilon_decay = 0.995
    self.gamma = 0.95
    self.batch_size = 32
    self.max_treward = 0
    self.averages = list()
    self.memory = deque(maxlen=2000)
    self.osn = env.observation_space.shape[0]
    self.model = self.create_model()

  def create_model(self):
    model = Sequential()
    model.add(Dense(24, input_dim=self.osn, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(env.action_space.n, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return env.action_space.sample()
    action = self.model.predict(state)
    return np.argmax(action)
  
  def replay(self):
    batch = random.sample(self.memory, self.batch_size)
    for state, action, reward, next_state, done in batch:
      if not done:
        reward += self.gamma * np.amax(self.model.predict(next_state)[0])
      target = self.model.predict(state)
      target[0, action] = reward
      self.model.fit(state, target, epochs=1, verbose=False)
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

  def learn(self, episodes=1000):
    trewards = []
    for e in range(episodes):
      state = env.reset()
      state = np.reshape(state, [1, self.osn])
      for time in range(5000):
        action = self.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, self.osn])
        self.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
          treward = time + 1
          trewards.append(treward)
          av = sum(trewards[:25]) / 25 # Work out a Rolling average
          self.averages.append(av)
          self.max_treward = max(self.max_treward, treward)
          print('Episode: {}/{}, treward: {}, av: {:.2}, max: {:2}'.format(e, episodes, treward, av, self.max_treward))          
          break
      if av > 199 and self.finish:
        print('Completed after {} episodes'.format(e))
        break
      if len(self.memory) > self.batch_size:
        self.replay()

  def test(self, episodes=100):
    trewards = []
    for e in range(episodes):
      state = env.reset()
      state = np.reshape(state, [1, self.osn])
      for time in range(5000):
        env.render()
        action = np.argmax(self.model.predict(state)[0])
        next_state, reward, done, _ = env.step(action)
        state = next_state
        if done:
          treward = time + 1
          trewards.append(treward)
          print('Episode: {}/{}, treward: {}'.format(e, episodes, treward))
          break
    env.close()

agent = DQNAgent()

2024-05-29 22:20:38.413665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
